# Heston data cook

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## General 2d SDE

In [2]:
'''=========
sde_2d class init
=========='''

class SDE_2d:
    def __init__(self,
                init_state = [100., .04],
                drift = lambda x: [0.01, 0.03], #x: 2-d array
                vol = lambda x: [[0.1, 0], [0,0.1]], #x: 2-d array, vol: 2 by 2 array
                ):
        self.init_state = init_state
        self.drift = drift
        self.vol = vol
        

In [3]:
'''================
euler_2d_difference 
input:
    xh_i: 2_d current state
    dt: 1_d time diff
    dw: 2_d bm increase
output:
    xh_i_diff: 2_d increase for euler solution
=================='''

def euler_2d_diff(self, xh_i, dt, dw):
    #set SDE param
    mu = self.drift
    sigma = self.vol
    
    return np.array(mu(xh_i))*dt + np.matmul(sigma(xh_i), dw)

SDE_2d.euler_diff = euler_2d_diff

In [4]:
'''==========
euler method as a method of SDE_2d
input:
    time grid: np.array (t_i: i = 0, 1, ..., n)
output: 
    euler solution: np.array (Xh_i: i = 0, 1, ..., n)
==========='''

def euler_2d(self, grid):
    #step_size dt
    dt = np.diff(grid)
    
    #generate 2d bm icrement
    dw = np.array([np.random.normal(0, np.sqrt(dt)) for i in range(2)])

    #initialize euler solution
    x0 = self.init_state #2_d array
    xh = np.array([x0[i] + np.zeros(grid.shape) for i in range(2)])
 
    
    #run euler
    for i in range(dt.size):
        xh[:,i+1] = xh[:,i] + self.euler_diff(xh[:,i], dt[i], dw[:,i]) #euler iteration        
    return xh

SDE_2d.euler = euler_2d

## Heston subclass 

In [5]:
'''============
Heston class inherited from sde_2d
============='''

class Heston(SDE_2d):
    def __init__(self,
                 init_state = [100., .04],
                 r = .05,
                 kappa = 1.2,
                 theta = .04,
                 xi = .3,
                 rho = .5,
                ):
        self.init_state = init_state
        rho_bar = np.sqrt(1 - rho**2)
        self.drift = lambda x: [r*x[0], kappa*(theta -x[1])]
        self.vol = lambda x: [
            [np.sqrt(x[1])*x[0], 0], 
            [xi*np.sqrt(x[1])*rho, xi*np.sqrt(x[1])*rho_bar]
        ]
                              

In [6]:
'''================
euler_2d_difference for Heston class to overide SDE_2d class member

input:
    xh_i: 2_d current state
    dt: 1_d time diff
    dw: 2_d bm increase
output:
    xh_i_diff: 2_d increase for euler solution
=================='''

def euler_heston_diff(self, xh_i, dt, dw):
    #set SDE param
    mu = self.drift
    sigma = self.vol
    xh_i[1] = np.max([0, xh_i[1]]) #new line to override
    
    return np.array(mu(xh_i))*dt + np.matmul(sigma(xh_i), dw)

Heston.euler_diff = euler_heston_diff #override

## European options

Test Euler method: 
We implement ordinary MC to find BSM call price, compare with exact price, see [pdf](../doc/euler_sde_2d.pdf)

In [7]:
'''=========
European option class init
=========='''
class EuropeanOption:
    def __init__(self,
                otype = 1, # 1: 'call'
                        # -1: 'put'
                strike = 110.,
                maturity = 1.
                ):
        self.otype = otype
        self.strike = strike
        self.maturity = maturity
        
    def payoff(self, s): #s: excercise price
        otype = self.otype
        k = self.strike
        maturity = self.maturity
        return np.max([0, (s - k)*otype])
        



In [8]:
'''===================
add 
euler price engine method for european option 
to
Heston class
===================='''

def euler_price(self, euro_option, num_timestep=10, num_path=1000):
    r = self.drift([1,1])[0] #interest rate
    time_grid = np.linspace(0, euro_option.maturity, num_timestep)
    terminal_price = [self.euler(time_grid)[0, -1] for i in range(num_path)]
    payoff = [euro_option.payoff(s) for s in terminal_price]
    return np.exp(-r*euro_option.maturity)*np.average(payoff)
    
Heston.euler_price = euler_price    

In [9]:
#Test call price

heston1 = Heston()
option1 = EuropeanOption(otype = 1, strike = 100., maturity = 1.)
print('call price is ' + str(heston1.euler_price(option1)))

call price is 10.286980263787122


In [10]:
#Test put price

heston2 = Heston()
option2 = EuropeanOption(otype = -1, strike = 100., maturity = 1.)
print('put price is ' + str(heston1.euler_price(option2)))

put price is 5.557643923860924


## Cook data for European options

In [11]:
'''=================
Heston class to be used
===================='''
init_state = [100., .04]
r = .03
kappa = 1.2
theta = .04
xi = .3
rho = .5
heston3 = Heston(init_state,
                 r,
                 kappa,
                 theta,
                 xi,
                 rho
                )


In [12]:
'''=============
Write option prices into a file
================'''

header1 = (
    '# Heston (init_state = ' 
    + str(init_state)
    + ', r = '
    + str(r)
    + ', kappa = '
    + str(kappa)
    + ', theta = '
    + str(theta)
    + ', xi = '
    + str(xi)
    + ', rho = '
    + str(rho)
    + ')'
    +'\n'
          )
header2 = '#otype, maturity, strike, price \n'
with open('optiondata1.dat','w') as file_handle:
    file_handle.write(header1)
    file_handle.write(header2)
    for otype in [-1,1]:
        for maturity in np.array([1, 2, 3, 5, 6, 12])/12:
            for strike in heston3.init_state[0]+np.arange(-4, 5):
                option3 = EuropeanOption(otype, strike, maturity)
                file_handle.write(str(otype) + ',' 
                                  +str(maturity) + ',' + str(strike) + ',' 
                                  + str(heston3.euler_price(option3, num_timestep = 10 + np.int(+maturity*12*5))) +'\n')
file_handle.close()               

In [13]:
#Read data
np_option_data1 = np.loadtxt('optiondata1.dat', comments='#', delimiter=',')
print(np_option_data1[0:5, :])

[[-1.00000000e+00  8.33333333e-02  9.60000000e+01  6.85769726e-01]
 [-1.00000000e+00  8.33333333e-02  9.70000000e+01  1.02374103e+00]
 [-1.00000000e+00  8.33333333e-02  9.80000000e+01  1.24578209e+00]
 [-1.00000000e+00  8.33333333e-02  9.90000000e+01  1.69887134e+00]
 [-1.00000000e+00  8.33333333e-02  1.00000000e+02  2.24362754e+00]]


In [14]:
np_option_data2 = np.loadtxt('optiondata1.dat', comments='#', delimiter=',')

filter1 = np_option_data2[np_option_data2[:,1] == 2/12]
filter2 = filter1[filter1[:,0] == 1.]
filter3 = filter2[filter2[:,2].astype(int)%2 == 1]

In [15]:
filter4 = np_option_data2[np_option_data2[:,1] == 5/12]
filter5 = filter4[filter4[:,0] == 1.]
filter6 = filter5[filter5[:,2].astype(int)%2 == 1]

In [16]:
filter7 = np.append(filter3, filter6, axis = 0)

In [17]:
np.savetxt('optiondata2.dat', filter7, delimiter = ',', header = header2, footer = header1)

In [18]:
np.loadtxt('optiondata2.dat', comments='#', delimiter=',')

array([[  1.        ,   0.16666667,  97.        ,   5.32705461],
       [  1.        ,   0.16666667,  99.        ,   3.86224255],
       [  1.        ,   0.16666667, 101.        ,   2.7204371 ],
       [  1.        ,   0.16666667, 103.        ,   2.1202793 ],
       [  1.        ,   0.41666667,  97.        ,   7.23756307],
       [  1.        ,   0.41666667,  99.        ,   5.95053461],
       [  1.        ,   0.41666667, 101.        ,   5.2640122 ],
       [  1.        ,   0.41666667, 103.        ,   4.97493422]])